In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('greattit.csv').drop(['Unnamed: 0', 'Genus', 'Specific_epithet', 'Subspecies', 'English_name', 'Recordist', 
                                        'Locality', 'Audio_file', 'License', 'Url', 'Date'], axis=1)
data.head()

,Recording_ID,Country,Latitude,Longitude,Vocalization_type,Quality,Time
0,441880,Netherlands,52.1468,5.3788,alarm call,no score,09:00
1,440223,France,42.6080,8.8610,call,no score,09:30
2,440205,Netherlands,52.1331,5.0489,begging call,no score,14:00
3,439558,France,49.6682,-1.4283,song,no score,18:00
4,439316,France,42.6080,8.8610,call,no score,11:00


In [3]:
data = data.dropna(subset=['Vocalization_type'])

In [4]:
binary_data = data.loc[data.Vocalization_type.str.contains('song') != data.Vocalization_type.str.contains('call')]
binary_data['labels'] = binary_data.Vocalization_type.str.contains('call').astype('int')

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
binary_data['labels'].sum() / len(binary_data)

0.6265389876880985

In [6]:
from scipy.stats import skew
import os
import librosa

def get_mfcc(name, path):
    try:
        b, _ = librosa.core.load(path + str(name) + '.wav')
    except:
        try:
            b, _ = librosa.core.load(path + '-' + str(name) + '.wav')
        except:
            return pd.Series([0] * 425)
        
    ft1 = librosa.feature.mfcc(b, n_mfcc=80)
    ft2 = librosa.feature.zero_crossing_rate(b)[0]
    ft3 = librosa.feature.spectral_rolloff(b)[0]
    ft4 = librosa.feature.spectral_centroid(b)[0]
    ft5 = librosa.feature.spectral_contrast(b)[0]
    ft6 = librosa.feature.spectral_bandwidth(b)[0]
    ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.min(ft1, axis = 1)))
    ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.min(ft2)))
    ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.min(ft3)))
    ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.min(ft4)))
    ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.min(ft5)))
    ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.min(ft6)))
    return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))

In [7]:
train_data = pd.DataFrame()
train_data['Recording_ID'] = binary_data['Recording_ID']

In [8]:
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()
train_data = train_data['Recording_ID'].progress_apply(get_mfcc, path='./sounds/')

100%|██████████| 3655/3655 [5:23:18<00:00,  5.01s/it]


In [12]:
train_data['id'] = binary_data['Recording_ID']
train_data['labels'] = binary_data['labels']
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,417,418,419,420,421,422,423,424,id,labels
0,-436.514074,127.301726,18.283442,-6.818973,-0.517332,17.952541,10.912547,12.669348,6.394881,10.341830,...,0.726950,23.723413,7.819720,2475.112147,213.669915,2.583432,3428.771732,2161.574903,441880,1
1,-239.074620,19.287065,-68.560588,-18.418223,-21.925773,-7.657923,-15.589265,-4.075395,-15.764188,-4.112783,...,1.208634,50.550359,4.978101,2434.314219,269.386657,-1.763712,3057.348932,1179.227306,440223,1
2,-367.704830,-31.474539,-108.760128,-90.779177,-75.426763,-23.981639,-25.067120,11.283314,3.798569,27.655528,...,0.626702,23.814826,1.402700,2309.327586,215.807624,-0.914716,2801.966575,1618.665418,440205,1
3,-508.650269,-38.546771,-95.776263,4.488206,-46.348091,-5.754554,-7.404113,-8.611265,-5.022680,3.629960,...,0.300264,38.912805,0.829720,1789.832920,508.838089,-0.666589,2693.892107,660.609600,439558,0
4,-321.269150,-76.923236,-119.413991,-13.216269,-21.198838,-4.078821,-5.427807,-1.305652,1.698714,5.166934,...,-0.470492,39.343142,3.233353,2138.560642,441.305899,-1.505895,2716.800178,872.997235,439316,1


In [ ]:
train_data.to_csv('train_data.')